### Prompt engineering

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
import torch

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [ ]:
def get_llm_response(prompt: str) -> dict:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=200)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
prompt = "I want to ask Mistral LLM to generate 5 domains for a business, based on a business description, such as 'pet grooming'. What would be the best prompt so it would generate for example 'HappyTails.com', 'HappyPaws.com', 'FurCare.com' and similar?"
answer = get_llm_response(prompt)
answer

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["I want to ask Mistral LLM to generate 5 domains for a business, based on a business description, such as 'pet grooming'. What would be the best prompt so it would generate for example 'HappyTails.com', 'HappyPaws.com', 'FurCare.com' and similar?",
 '',
 'To generate a list of domain names for a pet grooming business, you can use the following prompt:',
 '',
 '"Generate 5 unique domain names for a pet grooming business. The names should be catchy, easy to remember, and relevant to the pet grooming industry. The domain extensions can be .com, .net, .org, or .co."',
 '',
 'This prompt should help Mistral LLM generate a list of domain names that are suitable for a pet grooming business. Here are some examples of the generated domain names:',
 '',
 '* HappyTails.com',
 '* FurryFriends.net',
 '']

In [ ]:
business_description = "baking and selling pies"

In [ ]:
# zero shot prompt
prompt = f"Suggest 5 creative domain names in csv format for a business with a description of: {business_description}"
print(get_llm_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Suggest 5 creative domain names in csv format for a business with a description of: baking and selling pies and pastries', '', '"business_name,domain_name', '1. SweetPieEmporium.com, sweetpieemporium.com', '2. FlakyFingersBakery.com, flakyfingersbakery.com', '3. PiePassionate.com, piepassionate.com', '4. CrustCrafters.com, crustcrafters.com', '5. PastriesByPastryQueen.com, pastriesbypastryqueen.com"']


In [ ]:
# few shot prompt
prompt = f"If a business description is 'pet grooming' then their domains could be 'HappyTails.com', 'FurCare.com', 'HappyPaws.com' and similar. If a business description is 'web hosting services' then their domain could be 'hostinger.com'. Suggest 5 unique, short and memorable domain names in csv format for a business with a description of: '{business_description}'"
print(get_llm_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["If a business description is 'pet grooming' then their domains could be 'HappyTails.com', 'FurCare.com', 'HappyPaws.com' and similar. If a business description is 'web hosting services' then their domain could be 'hostinger.com'. Suggest 5 unique, short and memorable domain names in csv format for a business with a description of: 'baking and selling pies'", '', '"PieParadise.com", "SweetCrust.com", "PieMaker.com", "PieEmporium.com", "PieNirvana.com"']


In [ ]:
# LLM generated prompt
prompt = f"Generate 5 unique domain names in for a {business_description} business. The names should be catchy, easy to remember, and relevant to the {business_description} industry. provide names in csv format."
print(get_llm_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Generate 5 unique domain names in for a baking and selling pies business. The names should be catchy, easy to remember, and relevant to the baking and selling pies industry. provide names in csv format.', '', '"PiePerfection.com", "SweetPieSensations.com", "PieParadise.com", "CrispyCrustCreations.com", "PiePassionate.com"']
